# Report of Project 05: Implementation and Evaluation of Machine Learning (Support Vector Machine) Segmentation
## Data analysis project - B.Sc. Molecular Biotechnology Heidelberg University
### 19.07.21
### Authors: Michelle Emmert, Juan Hamdan, Laura Sanchis Pla and Gloria Timm

*ich werde kommentare, anmerkungen, noch zu klärende fragen... immer in kursiv schreiben*

*Idee: unseren algorithmus anhand eines bildes erklären und dieses immer wieder im report zeigen um die veränderungen
zu zeigen & am ende dice berechnen --> veranschaulichen*

*Unterüberschriften noch einfügen*

# Abstract

Support Vector Machines, also known as SVMs, are supervised learning models with associated learning algorithms that
analyze data  for classification and regression analysis.

Their theoretical foundations and their experimental success encourage further research on their characteristics, as well
as their further use. This report presents the summary of a project that implement and evaluate Support Vector Machine
Segmentation for segmentation of cell nuclei using dice score, synthetically generated images, and pre-processing methods.




# Table of contents *ggf. noch 2.1, 2.2 etc.*
**1. Introduction** <br>
**2. Our Data** <br>
**3.** <br>
**4. Pre-processing** <br>
**5. Implementation of the support vector machine** <br>
**6. Evaluation using the Dice coefficient** <br>
**6.1 The Theory Behind the Dice Coefficient** <br>
**6.2 Implementing the dice coefficient** <br>
**6.3 Synthetic Images** <br>
**6.4 PCA for feature selection for the SVM** <br>
**6.5 Stochastic gradient decent to minimize the loss gradient** <br>
**6.6 Cross-validation
**7. Results** <br>
**8. Discussion** <br>
**9. Bibliography**


# 1. Introduction
Image segmentation is a process during which important features of a picture are extricated, to aid analysis and extraction
of information. This is done by assigning labels to all pixels of the image. Pixels sharing defined traits, are allocated the same label.
Nuclei segmentation is a subform of image segmentation. In medicine nuclei segmentation could be/is used to classify and
grade cancer based on cancer histopathology. Today, cancer grading is still often done manually by visual analysis of tissue samples.
Problematic with this method is its inter- and intra-observer variability regarding the gradings quality, its low reproducibility
as well as its great time consumption. Cancer grading is an essential step in quantifying the degree of malignancy and thus
to predict patient prognosis and prescribe a treatment.

Using a support vector machine is one way nuclei segmentation can be achieved.
We use the support vector machine to differentiate between foreground and background pixels. Foreground pixels are
subsequently labeled as nuclei and displayed white. Background pixels are colored black. This also allows counting of the nuclei in the end.
To efficiently segment our images with a support vector machine, they first need to undergo preprocessing, to enhance
picture quality. After segmentation, we want to evaluate the quality of our segmentation, which we will be using the Dice
score for.

# 2. Our Data
Our data consists of 3 datasets with a total of 28 very different kinds of images all showing nuclei.
The pictures of the first dataset show GFP-transfected GOWT1 mouse embryonic stem cells. The second set of images show
Histone-2B-GFP expressing HeLa cells, while our third set consists of pictures of mouse embryonic fibroblasts, in which
CD-antigens were tagged with enhanced-GFP.
Though all images are microscopic images of nuclei, the three sets vary greatly in other features.
The images are of different formats (1024 x 1024, 1100 x 700, 1344 x 1024), were all acquired differently and display
different numbers of nuclei: within a range of 15-65 nuclei per image.
Additionally, all images pose diverse challenges for our image analysis.
Firstly brightness and resolution of the images differ. On top of that every set of images has its individual challenges
like white flashes, clustering of nuclei or nuclei leaving the image or undergoing mitosis.


# 6. Dice Coefficient
## 6.1 The Theory Behind the Dice Coefficient
The Dice coefficient is a score to evaluate and compare the accuracy of a segmentation.
Needed for its calculation are the segmented image, as well as a corresponding binary reference point also called
ground truth (Bertels et al., 2019).
As a ground truth image, researchers mostly use the segmentation result of humans. We will use the ground truth images
provided with our data sets, which we suspect to be acquired by this method.
Using the ground truth image, the labels true positive (TP), false positive (FP) and false
negative (FN) are assigned to each pixel of the segmented image (Menze et al., 2015).
This information is then used to calculate the dice coefficient using formula (1):

\begin{equation}
(1) \ dice = {\frac{2TP}{2TP + FP + FN}} \ \ \varepsilon \ \ [0,1]
\end{equation} <br>
(Menze et al., 2015)

A dice score of 0 indicates that the ground truth and the segmentation result do not have any overlap. A dice score of 1 on the
other hand, shows a 100% overlap of ground truth and segmented image (Bertels et al., 2019).


## 6.2 Implementing the dice coefficient

In [1]:
import dicescore_code.dicescore as dicescore


ModuleNotFoundError: No module named 'Finalmodules.dicescore'

## 6.3 Synthetic Images
### 6.3.1 Definition and Goal
The concept behind creating synthetic images, is to use algorithms and images, which are already available to generate
new ones. (Dunn et al., 2019) Although our first objective was to just use these new images to test our code for the dice score, we realized
while researching for this topic that synthetic images have an immense potential, most of all for the training of machine
learning algorithms. By expanding our training set with diverse images of good quality, we expect a more accurate model (Mayer et al., 2017 ALTERNATIV: https://arxiv.org/abs/1807.07428).
In order to train our model and test it afterwards with new data, we have to split up our dataset of 28 cell images
between those two sets, which leads to a training set of less than 28 images. Because of this, we decided to implement
our synthetically produced images not only to test the dice score, but to enlarge our training data pool for our
Support Vector Machine, and afterwards, check if its efficiency was improved with our dice score. There are many methods
that can be used in order to generate synthetic images (Ward et al., 2019). Because of the scope of our project and the
kind of images we want to produce, we focused on image composition and domain transfer.

### 6.3.2 Image Composition
Image composition consists of taking various foreground images, which have been segmented out of their backgrounds or
have a .png format to begin with, and paste them onto different backgrounds (Tripathi, 2019). The foreground images can
be modified using different light conditions, contrasts, zooms or rotations in order to achieve more variety in the results
(Ward et al., 2019; Alghonaim and Johns, 2020).
--> probably not as useful for our case as cells are usually in front of dark background, but still an option to evaluate
--> will the Dice Score get better with that method?

*Here we could insert the code from my Jupyter notebook -> issue: it hasn't been written by us
(and it hasn't been modified either...), so maybe it would be better to just use our own code when we write it.*

### 6.3.3 Domain randomization
In domain randomization there is a model from the object class you want to train your model for, and in that model,
every parameter from the object and its environment that is not necessary for its recognition by the machine has been
randomized (Mehta et al., 2020). This means for example size, lighting or color, and there are very powerful tools to do this, like Unity or
Blender (Ward et al., 2019; Alghonaim and Johns, 2020). <br>
*--> probably more useful, as we have changes in size, dividing or leaving cells etc. However, it is usually
used to train robots to work from a simulation to reality, so it might be a bit too much.*

# Support Vector Machine
In 1992, Vapnik and coworkers proposed a supervised algorithm, developed from statistical learning, to solve classification
problems (Vapnik et al., 1992). Since then, their machine learning method evolved into what is now known as
Support Vector Machines (SVM): a class of algorithms for classification, regression and other applications that
represent the current state of the art in the field (Suthaharan, 2016; Guanasekaran, 2010; Christianini and Ricci, 2008).

By providing a training data set with binary labels, the SVM is able to learn how to classify data points using certain
features. This capability can subsequently be used to classify new data, called test data, using its features (Thai et. al, 2012).
SVMs have been successfully applied to a number of applications, ranging from the time series prediction and face recognition
to biological data processing for medical diagnosis (Evgeniou, 2001).
In image processing, support vector machines are used for one of the classical challenges: image classification (Evgeniou, 2001).

## The Mathematical Background

The mathematical concepts are the key in understanding how support vector machines work.
The goal of a support vector machine is to seperate data points into two groups of provided labels with an optimal
hyperplane.
This hyperplane is described by

\begin{equation}
(2) \ w * x + b = 0
\end{equation}

and fullfilling the following condition

\begin{equation}
(3) \ h =
\left\{
  \begin{aligned}
    +1 \ \ if \ \ w⋅x_i +b≥ +1 - \varepsilon_i\\
    -1 \ \ if \ \ w⋅x_i +b< -1 - \varepsilon_i
  \end{aligned}
  \right.
  \ \ \varepsilon \geqq 0 \ \forall_i \ ; \ i=1...m
\end{equation}

whilst for two dimensions $w = (a, -1)$, whereas $a$ is the slope of the line, and $x = (x_1, x_2)$ and represents a
data point. $\varepsilon$ is a variable, representing the inaccuracy of the hyperplane. It is added to the constraint to
prevent overfitting of the model onto the training set. Without $\varepsilon$ the geometric margin M is called a hard margin,
as it does not allow data points of one group to be falsely labeled as members of the other group. This does not result in
the best model, as single falsely assigned data points, can have a lower impact on the quality of the model, than a suboptimal
hyperplane. Therefore, $\varepsilon$ is introduced and therefore M named a soft margin.

To choose the optimal hyperplane we want to minimize the margin as follows.

\begin{equation}
(4)\ M = min_{i=1...m} \ y_i(\frac{w}{||w||}*x + \frac{b}{||w||})
\end{equation}

The largest margin M out of all margins computed in our training phase will be selected. The variables w and b are
divided through the length of the vector w calculated with the Euclidean norm formula, as they have to be scale invariant.
These variables w and b are the ones to be found a value for.

This leads us to the following optimization problem.
We want to maximize M:

\begin{equation}
(5) \ max_{w,b,\varepsilon} M
\end{equation}

This maximization problem is equivalent to

\begin{equation}
(6) \ max_{w,b,\varepsilon} \frac{1}{||w||} \ + \ \sum^{m}_{i=1}\varepsilon_i
\end{equation}

and can be rewritten as the following minimalization problem.

\begin{equation}
(7) \ min_{w,b,\varepsilon}  \ \frac{1}{2}||w||^2\ + \ C\sum^{m}_{i=1}\varepsilon_i \\
subject \ to \ \  y_i(w⋅x_i+b)≥1− \varepsilon_i \ , \varepsilon \geqq 0 \ \forall_i \ , \ i=1...m
\end{equation}

The regularization parameter C is chosen by the user and determines the weight of $\varepsilon$.
The larger C is, the higher the penalty for errors and therefore the harder the margin.

In order to solve this constrained optimization problem, in which we want to maximize the margin while fulfilling our
 conditions or constraints, Lagrange multipliers are used. The principle behind it, is that at the optimum, the gradient
 of our objective function is parallel or antiparallel to the gradient of the constraint function. Because of this,
 they have to be equal or a multiple of each other, which is what the Lagrange multiplier is showcasing.

\begin{equation}
(8) \ \nabla f(x) - \alpha \nabla g(x) = 0
\end{equation}

When we insert our functions, we get the following Lagrangian function:

\begin{equation}
(9) \ f(x)= \frac{1}{2}||w||^2\ + \ C\sum^{m}_{i=1}\varepsilon_i
\\(10) \ g(x) = y_i(w⋅x_i+b) - 1 + \varepsilon_i
\\(11) \ \mathcal{L}(w,b,\alpha) =\frac{1}{2}||w||^2\ + \ C\sum^{m}_{i=1}\varepsilon_i - \sum^{m}_{i=1}\alpha_i[y_i
(w⋅x_i+b) - 1 + \varepsilon_i]
\end{equation}

In order to solve this Lagrange problem, it is relaxed into a dual problem, where the constraints are incorporated
into the function, so that it only depends on the Lagrange multipliers, making it easier to be solved. These are the
two constraints for the dual problem:

\begin{equation}
(12) \ \nabla_w \mathcal{L}(w,b,\alpha) = w - \sum^{m}_{i=1} \alpha_i y_i x_i = 0
\\ (13) \ \nabla_b \mathcal{L}(w,b,\alpha) = - \sum^{m}_{i=1} \alpha_i y_i = 0
\end{equation}

If they are inserted into the Lagrange function, this is the result for the dual problem.
\begin{equation}
(14) \ max_{\alpha}  \ \sum^{m}_{i=1}\alpha_i - \frac{1}{2}\sum^{m}_{i=1}\sum^{m}_{j=1}\alpha_i\alpha_j y_i y_j x_i · x_j \\
subject \ to \ \ 0≤\alpha_i≤C, i=1...m, \sum^{m}_{i=1}\alpha_iy_i = 0
\end{equation}

From the above equation it becomes clear that its maximization only depends on the dot product of the support vectors
$x_i · x_j$. This becomes an advantage when the data are not linearly separable. The trick is to transform the data
into a higher dimension, where a hyperplane can be found that separates it. However, if the dataset is very big,
calculating the transformation would be very time-consuming. Because of this, the Kernel trick is used. It consists
in a function which calculates the dot product $x_i · x_j$ as if it were in a higher dimension. There is the linear
Kernel function

\begin{equation}
K(x_i, x_j)=\phi(x_i) · phi(x_j)
\end{equation}

but it varies depending on the type of data that has to be classified. Other know and comonly used Kernel functions are:

\begin{equation}
K(x_i, x_j)=x_i · x_j,
\end{equation}

!!!*still missing/ has to be elaborated*!!!:
-Wolfe dual problem
-Kernel trick

## 6.5 K-Fold Cross validation
Validation is a widely used technique in data science to evaluate how stable a machine learning (ML) model is and to
assess how well the model would generalize to new, independent data. Relevant for these two characteristics is the MLs
ability to differentiate between relevant patterns and noise in the data available (Vabalas et. al, 2019). As a measure for how good the ML is
able to achieve this, the bias-variance tradeoff can be used (Geman et. al, 1992; Berrar, 2019).
Bias and variance are both sources of error in ML generalization. With increasing model complexity, bias decreases and
variance increases. In short: High bias indicates 'underfitting', high variance points at an 'overfitted' model.
'Underfitting' describes the performance a model, which is neither able to classify its training data nor new data well,
because it captures too little patterns. 'Overfitting' on the other hand refers to a model, that is overly sensitive to
noise and random pattern in it's training data and for that reason performs poorly on new data. Optimally both bias and
variance could be minimized (Geman et. al, 1992; Berrar, 2019). In reality the right balance of both is needed to generate an optimal model. (QUELLE)

One validation technique is k-fold cross validation (CV).
In CV, the data available is split into k subsets. The data encompasses n dissimilar samples. k is a random
integer between 1 and n. For each iteration k-1 subsets are used as training data, while the remaining subsets are
used to test the model. To put it differently: each data sample is part of the testing data once and part of the training
data for all other iterations (Vabalas et. al, 2019).

As our data consists of only 28 images, we used a special case of cross-validation called leave one out cross-validation (LOOCV).
In this approach k equals the number of samples.

*This approach substantially reduces bias, as it uses all data points. Simultaneously variance also decreases,
because YXXXX. But as only one datapoint is used for testing in each iteration, higher variation in testing model effectiveness
can occur.* (QUELLE)

#https://www.statology.org/bias-variance-tradeoff/
#https://www.researchgate.net/profile/Daniel-Berrar/publication/324701535_Cross-Validation/links/5cb4209c92851c8d22ec4349/Cross-Validation.pdf

**9. Bibliography**
Alghonaim, R., Johns, E. (2020).Benchmarking Domain Randomisation for Visual Sim-to-Real Transfer. CoRR.

Berrar, D. (2019). Cross-validation. Data Science Laboratory, Tokyo Institute of Technology.

Bertels, J., Eelbode, T., Berman, M., Vandermeulen D., Maes F., Bisschops, R., Blaschko, M. (2020).
Optimization for Medical Image Segmentation: Theory and Practice when evaluating with Dice Score or Jaccard Index.
IEEE Trans Med Imaging.

Boser, B., Guyon, I., Vapnik, V. (1992). A training algorithm for optimal margin classifiers. Proceedings of the fifth
annual workshop on Computational learning theory. Ed. 07.1992, 144–152.

Christianini, N., Ricci, E. (2008). Support Vector Machines. Encyclopedia of Algorithms, Springer.

Dunn, K.W., Fu, C., Ho, D.J., Lee S., Han S., Salama P., Delp E. (2019). DeepSynth: Three-dimensional nuclear segmentation of
biological images using neural networks trained with synthetic data. Sci Rep 9, 18295

Evgeniou, T., Pontil, M. (2001). Support Vector Machines: Theory and Applications. Computer Science

Guanasekaran, T., Shankar Kumar, K.R. (2010), Modified concentric circular micostrip array configurations for
wimax base station. Journal of Theoretical and Applied Information Technology Vol 12. No. 1

Mayer, N., Ilg, E., Fischer, P., Hazirbas, C., Cremers, D., Dosovitskiy, A.,Brox, T. (2018). What Makes Good Synthetic
Training Data for Learning Disparity and Optical Flow Estimation?. Int J Comput Vis 126, 942–960.

Mehta, B., Diaz, M., Golemo, F., Pal, C. J., Paull, L. (2020). Active Domain Randomization. Proceedings of Machine Learning Research.

Menze, B., Jakab, A., Bauer, S., Kalpathy-Cramer, J., Farahani, K., Kirby, J., Burren, Y., Porz, N., Slotboom, J., Wiest, R., Lanczi, L.,
Gerstner, E., Weber, M., Arbel, T., Avants, B., Ayache, N., Buendia, P., Collins, D., Cordier, N., Corso, J., Criminisi, A., Das, T.,
Delingette, H., Demiralp, Ç., Durst, C., Dojat, M., Doyle, S., Festa, J., Forbes, F., Geremia, E., Glocker, B., Golland, P., Guo, X., Hamamci, A.,
Iftekharuddin, K., Jena, R., John, N., Konukoglu, E., Lashkari, D., Mariz, J., Meier, R., Pereira, S., Precup, D., Price, S., Raviv, T.,
Reza, S., Ryan, M., Sarikaya, D., Schwartz, L., Shin, H., Shotton, J., Silva, C., Sousa, N., Subbanna, N., Szekely, G., Taylor, T.,
Thomas, O., Tustison, N., Unal, G., Vasseur, F., Wintermark, M., Ye, D., Zhao, L., Zhao, B., Zikic, D., Prastawa, M., Reyes, M., Van Leemput, K. (2015).
The Multimodal Brain Tumor Image Segmentation Benchmark (BRATS). IEEE Trans Med Imaging.

Suthaharan S. (2016). Support Vector Machine. Machine Learning Models and Algorithms for Big Data Classification.
Integrated Series in Information Systems, vol 36. Springer

Geman, S., Bienenstock, E., Doursat, R. (1992). Neural Networks and the Bias/Variance Dilemma. Neural Computation.

Thai, L.H., Tran S.H., Nguyen T.T. (2012). Image Classification using Support Vector Machine and Artificial Neural Network.
International Journal of Information Technology and Computer Science(IJITCS)

Tripathi, S., Chandra S., Agrawal, A., Tyagi, A., Rehg, J. M., Chari, V. (2019). Learning to Generate Synthetic
Data via Compositing. IEEE Xplore.

Vabalas, A., Gowen, E., Poliakoff, E., Casson, A.J. (2019). Machine learning algorithm validation with a limited sample size. Plos one.

Ward, D.,Moghadam P.,Hudson, N. (2019). Deep Leaf Segmentation Using Synthetic Data. CoRR.

